# Simple Substitution
`w266 Final Project: Crosslingual Word Embeddings`

The code in this notebook was used to develop an algorithm to generate crosslingual word embeddings by training on a monolingual corpus and substituting translations at runtime.

# Notebook Setup

In [1]:
# general imports
import os
import re
import sys
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from __future__ import print_function
# tell matplotlib not to open a new window
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

In [44]:
# filepaths
BASE = '~/Documents/MIDS/w266/Data' #'/home/mmillervedam/Data'
PROJ = '~/Documents/MIDS/w266/FinalProject'#'/home/mmillervedam/ProjectRepo'
FPATH_EN = BASE + '/test/wiki_en_10K.txt' # first 10000 lines from wiki dump
FPATH_ES = BASE + '/test/wiki_es_10K.txt' # first 10000 lines from wiki dump
#FULL_EN = BASE + '/en/full.txt'
#FULL_ES = BASE + '/es/full.txt'
EN_ES_DICT = PROJ +'/XlingualEmb/data/dicts/en.es.panlex.all.processed'
EN_IT_DICT  = PROJ +'/XlingualEmb/data/dicts/en.it.panlex.all.processed'
EN_IT_RAW = PROJ + '/XlingualEmb/data/mono/en_it.shuf.10k'
EN_IT_RAW = "/Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k"

# Load & Preprocess Data
__`ORIGINAL AUTHORS SAY:`__ "Normally, the monolingual word embeddings are trained on billions of words. However, getting that much of monolingual data for a low-resource language is also challenging. That is why we only select the top 5 million sentences (around 100 million words) for each language." - _Section 5.1, Duong et. al._ 

In [21]:
from parsing import Corpus, Vocabulary, batch_generator

### Corpus

In [45]:
# load corpus
en_it_data = Corpus(EN_IT_RAW)

In [46]:
# Corpus Stats
!wc {EN_IT_RAW}

   20000  430928 3746786 /Users/mmillervedam/Documents/MIDS/w266/FinalProject/XlingualEmb/data/mono/en_it.shuf.10k


__`i.e.:`__ 20K sentences (10K in each language) with ~430K tokens
> So this must not be their full data For now, I'm just going to look at the top 20K words and see what happens. In reality we should probably modify the Vocab class so that it explicily collects the top words for each language separately and then concatenates the index.

### Dictionary

In [23]:
# loading english-italian dictionary
pld = pd.read_csv(EN_IT_DICT, sep='\t', names = ['en', 'it'], dtype=str)
en_set = set(pld.en.unique())
it_set = set(pld.it.unique())

In [24]:
# dictionary vocab lengths:
print('EN:', len(en_set))
print('IT:', len(it_set))

EN: 356410
IT: 346572


### Vocabulary

In [50]:
# train multilingual Vocabulary# create vocab
en_it_vocab = Vocabulary(en_it_data.gen_tokens(), size = 100000)

### CBOW Data Generator
__`check paper re: window size!`__

In [52]:
BATCH_SIZE = 48
WINDOW_SIZE = 48
MAX_EPOCHS = 1 # fail safe

In [53]:
batches = batch_generator(en_it_data, 
                          en_it_vocab, 
                          BATCH_SIZE, 
                          WINDOW_SIZE, 
                          MAX_EPOCHS)

In [54]:
for context, label in batches:
    print(context, label)
    break

[[0, 1], [0, 1], [34, 17318, 1638, 310, 5734, 7, 35, 3213, 60, 4449, 8437, 10, 17821, 7, 44, 6805, 20, 940, 10, 46246, 14232, 300, 41, 19895, 10, 737, 50, 7043, 7, 8057, 613, 20841, 48199, 29, 784, 4081, 2458, 7, 274, 41, 27292, 60, 4449, 8437, 11, 1], [17318, 1638, 310, 5734, 7, 35, 3213, 60, 4449, 8437, 10, 17821, 7, 44, 6805, 20, 940, 10, 46246, 14232, 300, 41, 19895, 10, 737, 50, 7043, 7, 8057, 613, 20841, 48199, 29, 784, 4081, 2458, 7, 274, 41, 27292, 60, 4449, 8437, 11, 1], [1638, 310, 5734, 7, 35, 3213, 60, 4449, 8437, 10, 17821, 7, 44, 6805, 20, 940, 10, 46246, 14232, 300, 41, 19895, 10, 737, 50, 7043, 7, 8057, 613, 20841, 48199, 29, 784, 4081, 2458, 7, 274, 41, 27292, 60, 4449, 8437, 11, 1], [310, 5734, 7, 35, 3213, 60, 4449, 8437, 10, 17821, 7, 44, 6805, 20, 940, 10, 46246, 14232, 300, 41, 19895, 10, 737, 50, 7043, 7, 8057, 613, 20841, 48199, 29, 784, 4081, 2458, 7, 274, 41, 27292, 60, 4449, 8437, 11, 1], [5734, 7, 35, 3213, 60, 4449, 8437, 10, 17821, 7, 44, 6805, 20, 940, 10